## Combining Datasets

Datasets we have
1. Fitness First
   1. Junction 10 (North) 4.5 rating, 284 reviews
   2. One George Street (Central/South) 4.4 rating, 283 reviews
   3. Tampines (East) 4.1 rating, 198 reviews
   4. Paya Lebar (East) 3.9 rating, 197 reviews
2. Virgin Active
   1. Tanjong Pagar (Central/South) 4.3 rating, 438 reviews
   2. Paya Lebar (East) 4.3 rating, 219 reviews

The dataframe outputs we want are
1. virgin_active_reviews
   1. virgin_active_positive_reviews
   2. virgin_active_negative_reviews
2. fitness_first_reviews
   1. fitness_first_positive_reviews
   2. fitness_first_negative_reviews
3. competitor_reviews

The other notebooks will analyze competitor gym reviews and ratings for better decision-making using the following NLP techniques
1.  Keyword Frequency Analysis over the Years
2.  Word Clouds by Rating: Compare word clouds for high (4-5 stars) and low (1-2 stars) ratings to identify key strengths and weaknesses.
3.  Sentiment by Topic: Analyze sentiment for specific topics (e.g., "equipment" or "classes") across different rating levels to pinpoint areas for improvement.
4.  Topic Modeling (LDA): Discover hidden themes (e.g., equipment, classes, staff) to understand common concerns and strengths.

In [160]:
import pandas as pd
import plotly.express as px

## Combine the Datasets

In [161]:
ff_junction = pd.read_csv('datasets/fitness-first-junction10.csv')
ff_george = pd.read_csv('datasets/fitness-first-georgest.csv')
ff_paya = pd.read_csv('datasets/fitness-first-paya.csv')
va_tanjong = pd.read_csv('datasets/virgin-active-tanjong.csv')
va_paya = pd.read_csv('datasets/virgin-active-paya.csv')

In [187]:
# We will use use pd.concat() to combine the dataframes row-wise
fitness_first_reviews = pd.concat([ff_junction, ff_george, ff_paya], axis=0, ignore_index=True)
virgin_active_reviews = pd.concat([va_paya, va_tanjong], axis=0, ignore_index=True)

In [182]:
fitness_first_reviews.head(2)

,place_id,place_name,review_id,name,reviewer_profile,rating,review_text,published_at,published_at_date,response_from_owner_text,response_from_owner_ago,response_from_owner_date,review_likes_count,total_number_of_reviews_by_reviewer,total_number_of_photos_by_reviewer,is_local_guide,review_translated_text,response_from_owner_translated_text
0,ChIJx6EEULwR2jER8Ps0iB4n4to,Fitness First @ Junction 10,ChdDSUhNMG9nS0VJQ0FnSUQ4cWNfODZBRRAB,Wellbase Builder Pte Ltd,https://www.google.com/maps/contrib/1111701812...,5,NaN,6 days ago,2024-12-11T14:43:14,NaN,NaN,NaN,0,11.0,72.0,1.0,NaN,NaN
1,ChIJx6EEULwR2jER8Ps0iB4n4to,Fitness First @ Junction 10,ChZDSUhNMG9nS0VJQ0FnSURQLTgzZExnEAE,Chiew Mei Ng,https://www.google.com/maps/contrib/1164550011...,1,Air In the gym very smelly gym never place air...,a week ago,2024-12-10T14:43:14,NaN,NaN,NaN,0,1.0,NaN,NaN,NaN,NaN


In [183]:
virgin_active_reviews.head(2)

,place_id,place_name,review_id,name,reviewer_profile,rating,review_text,published_at,published_at_date,response_from_owner_text,response_from_owner_ago,response_from_owner_date,review_likes_count,total_number_of_reviews_by_reviewer,total_number_of_photos_by_reviewer,is_local_guide,review_translated_text,response_from_owner_translated_text
0,ChIJgfCnragZ2jER9L5TfSSX0rg,Virgin Active Paya Lebar,ChdDSUhNMG9nS0VJQ0FnSUN2ak9QS2pnRRAB,Nas Radz,https://www.google.com/maps/contrib/1177844161...,5,The people at the counters are super nice & ac...,a day ago,2024-12-09T11:35:31,NaN,NaN,NaN,0,8.0,2.0,1.0,NaN,NaN
1,ChIJgfCnragZ2jER9L5TfSSX0rg,Virgin Active Paya Lebar,ChZDSUhNMG9nS0VJQ0FnSURQdGNXQ0tnEAE,Jade Sim,https://www.google.com/maps/contrib/1079996294...,5,Started trial pass this week and attended my f...,5 days ago,2024-12-05T11:35:31,NaN,NaN,NaN,0,14.0,NaN,NaN,NaN,NaN


##### Clean the Data and Feature Engineering (Updated)
1. Select the relevant columns: Keep only ['review_text', 'rating', 'published_at_date', 'place_name'].
2. Rename the 'review_text' column to review.
3. Add a new column 'length' that shows the length of each review.
4. Separate the 'place_name' column into two new columns: gym and branch.
5. Create a count plot to show the distribution of rating categories by year.
6. Keep the necessary columns: ['review', 'rating', 'gym', 'branch', 'length', 'year'].
7. Remove rows where the review column has null values.
8. Export the final DataFrame to CSV files: 'fitness-first-reviews.csv' and 'virgin-active-reviews.csv'.

## Fitness First

In [188]:
# 1. Select the relevant columns: Keep only ['review_text', 'rating', 'published_at_date', 'place_name'].
fitness_first_reviews = fitness_first_reviews[['review_text','rating','published_at_date','place_name']]

# 2. Rename the 'review_text' column to review.
fitness_first_reviews.rename(columns={'review_text': 'review'}, inplace=True)

# 3. Add a new column that shows the length of each review.
fitness_first_reviews.loc[:,'length'] = fitness_first_reviews['review'].astype(str).apply(len)

fitness_first_reviews.head(2)

,review,rating,published_at_date,place_name,length
0,NaN,5,2024-12-11T14:43:14,Fitness First @ Junction 10,3
1,Air In the gym very smelly gym never place air...,1,2024-12-10T14:43:14,Fitness First @ Junction 10,233


In [189]:
# 4. Separate the 'place_name' column into two new columns: gym and branch.

# Mapping for place_name to branch
branch_mapping = {
    'Fitness First Paya Lebar': 'Paya Lebar',
    'Fitness First @ Junction 10': 'Junction 10',
    'Fitness First - One George Street': 'One George Street'
}

# Create the 'branch' column based on the 'place_name' values
fitness_first_reviews.loc[:,'branch'] = fitness_first_reviews['place_name'].map(branch_mapping)

# Create the 'gym' column
fitness_first_reviews.loc[:,'gym'] = 'Fitness First'

fitness_first_reviews.head(2)

,review,rating,published_at_date,place_name,length,branch,gym
0,NaN,5,2024-12-11T14:43:14,Fitness First @ Junction 10,3,Junction 10,Fitness First
1,Air In the gym very smelly gym never place air...,1,2024-12-10T14:43:14,Fitness First @ Junction 10,233,Junction 10,Fitness First


In [194]:
# 5. Create a count plot to show the distribution of rating categories by year.

# Create a copy of the DataFrame to avoid modifying the original
fitness_first_reviews_copy = fitness_first_reviews

# Convert 'published_at_date' to datetime
fitness_first_reviews_copy['published_at_date'] = pd.to_datetime(fitness_first_reviews_copy['published_at_date'])

# Filter data to only include rows from 2017 onwards
fitness_first_reviews_copy = fitness_first_reviews_copy[fitness_first_reviews_copy['published_at_date'].dt.year >= 2017]

# Extract year from 'published_at_date'
fitness_first_reviews_copy.loc[:,'year'] = fitness_first_reviews_copy['published_at_date'].dt.year

# Create a count plot for rating categories by year
fig = px.histogram(fitness_first_reviews_copy, x='year', color='rating', 
                   title='Fitness First: Count of Rating Categories by Year',
                   labels={'year': 'Year', 'rating': 'Rating Category'},
                   category_orders={'rating': [1, 2, 3, 4, 5]})

# Show the plot
fig.show()

In [195]:
# Create the 'year' column
fitness_first_reviews['published_at_date'] = pd.to_datetime(fitness_first_reviews['published_at_date'])
fitness_first_reviews['year'] = fitness_first_reviews['published_at_date'].dt.year

# 6. Keep the necessary columns: ['review', 'rating', 'gym', 'branch', 'length', 'year'].
final_fitness_first_reviews = fitness_first_reviews[['review','rating','gym','branch','length','year']]
final_fitness_first_reviews

,review,rating,gym,branch,length,year
0,NaN,5,Fitness First,Junction 10,3,2024
1,Air In the gym very smelly gym never place air...,1,Fitness First,Junction 10,233,2024
2,This club doesnt even deserve one star rating ...,1,Fitness First,Junction 10,409,2024
3,I joined the centre last Oct. Staff here are r...,4,Fitness First,Junction 10,215,2024
4,Despite all sorts of brand names and competiti...,4,Fitness First,Junction 10,594,2024
...,...,...,...,...,...,...
759,Biggest disappointment of a gym. Only one squa...,1,Fitness First,Paya Lebar,386,2018
760,NaN,5,Fitness First,Paya Lebar,3,2018
761,NaN,5,Fitness First,Paya Lebar,3,2018
762,NaN,1,Fitness First,Paya Lebar,3,2018


In [ ]:
# 7. Remove rows where the review column has null values.
final_fitness_first_reviews = final_fitness_first_reviews.dropna(subset=['review'])
final_fitness_first_reviews.info()

In [152]:
# 8. Export the final DataFrame to 'fitness_first_reviews.csv'.
final_fitness_first_reviews.to_csv('filtered-datasets/fitness-first-reviews.csv', index=False)

## Virgin Active

In [196]:
# 1. Select the relevant columns: Keep only ['review_text', 'rating', 'published_at_date', 'place_name'].
virgin_active_reviews = virgin_active_reviews[['review_text','rating','published_at_date','place_name']]

# 2. Rename the 'review_text' column to review.
virgin_active_reviews.rename(columns={'review_text': 'review'}, inplace=True)

# 3. Add a new column that shows the length of each review.
virgin_active_reviews['length'] = virgin_active_reviews['review'].astype(str).apply(len)

virgin_active_reviews.head(2)

,review,rating,published_at_date,place_name,length
0,The people at the counters are super nice & ac...,5,2024-12-09T11:35:31,Virgin Active Paya Lebar,140
1,Started trial pass this week and attended my f...,5,2024-12-05T11:35:31,Virgin Active Paya Lebar,211


In [197]:
# 4. Separate the 'place_name' column into two new columns: gym and branch.

# Mapping for place_name to branch
branch_mapping = {
    'Virgin Active Paya Lebar': 'Paya Lebar',
    'Virgin Active Tanjong Pagar': 'Tanjong Pagar',
}

# Create the 'branch' column based on the 'place_name' values
virgin_active_reviews['branch'] = virgin_active_reviews['place_name'].map(branch_mapping)

# Create the 'gym' column
virgin_active_reviews['gym'] = 'Virgin Active'

virgin_active_reviews.head(2)

,review,rating,published_at_date,place_name,length,branch,gym
0,The people at the counters are super nice & ac...,5,2024-12-09T11:35:31,Virgin Active Paya Lebar,140,Paya Lebar,Virgin Active
1,Started trial pass this week and attended my f...,5,2024-12-05T11:35:31,Virgin Active Paya Lebar,211,Paya Lebar,Virgin Active


In [198]:
# 5. Create a count plot to show the distribution of rating categories by year.

# Create a copy of the DataFrame to avoid modifying the original
virgin_active_reviews_copy = virgin_active_reviews

# Convert 'published_at_date' to datetime
virgin_active_reviews_copy['published_at_date'] = pd.to_datetime(virgin_active_reviews_copy['published_at_date'])

# Filter data to only include rows from 2017 onwards
virgin_active_reviews_copy = virgin_active_reviews_copy[virgin_active_reviews_copy['published_at_date'].dt.year >= 2017]

# Extract year from 'published_at_date'
virgin_active_reviews_copy['year'] = virgin_active_reviews_copy['published_at_date'].dt.year

# Create a count plot for rating categories by year
fig = px.histogram(virgin_active_reviews_copy, x='year', color='rating', 
                   title='Virgin Active: Count of Rating Categories by Year',
                   labels={'year': 'Year', 'rating': 'Rating Category'},
                   category_orders={'rating': [1, 2, 3, 4, 5]})

# Show the plot
fig.show()

In [199]:
# Create the 'year' column
virgin_active_reviews['published_at_date'] = pd.to_datetime(virgin_active_reviews['published_at_date'])
virgin_active_reviews['year'] = virgin_active_reviews['published_at_date'].dt.year

# 6. Keep the necessary columns: ['review', 'rating', 'gym', 'branch', 'length', 'year'].
final_virgin_active_reviews = virgin_active_reviews[['review','rating','gym','branch','length','year']]
final_virgin_active_reviews

,review,rating,gym,branch,length,year
0,The people at the counters are super nice & ac...,5,Virgin Active,Paya Lebar,140,2024
1,Started trial pass this week and attended my f...,5,Virgin Active,Paya Lebar,211,2024
2,Zaidi is always very friendly with a warm welc...,5,Virgin Active,Paya Lebar,200,2024
3,Very good service. Especially when I lost my B...,5,Virgin Active,Paya Lebar,162,2024
4,Had a situation with one of my class bookings....,4,Virgin Active,Paya Lebar,195,2024
...,...,...,...,...,...,...
649,A big and spacious gym..,5,Virgin Active,Tanjong Pagar,24,2017
650,NaN,5,Virgin Active,Tanjong Pagar,3,2017
651,NaN,5,Virgin Active,Tanjong Pagar,3,2017
652,NaN,5,Virgin Active,Tanjong Pagar,3,2017


In [200]:
# 7. Remove rows where the review column has null values.
final_virgin_active_reviews = final_virgin_active_reviews.dropna(subset=['review'])
final_virgin_active_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 445 entries, 0 to 649
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  445 non-null    object
 1   rating  445 non-null    int64 
 2   gym     445 non-null    object
 3   branch  445 non-null    object
 4   length  445 non-null    int64 
 5   year    445 non-null    int32 
dtypes: int32(1), int64(2), object(3)
memory usage: 22.6+ KB


In [201]:
# 8. Export the final DataFrame to 'virgin_active_reviews.csv'.
final_virgin_active_reviews.to_csv('filtered-datasets/virgin-active-reviews.csv', index=False)